<a href="https://colab.research.google.com/github/vchai03/Synchronicity/blob/main/pose_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import cv2 as cv
from google.colab.patches import cv2_imshow
import math

In [ ]:
model_path = "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"
template_path = "standing2.jpeg"
target_path = "standing.jpeg"

In [ ]:
# Load TFLite model and allocate tensors (memory usage method reducing latency)
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

ValueError: ignored

In [ ]:
# Get input and output tensors information from the model file
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

In [ ]:
template_image_src = cv.imread(template_path)
template_image = cv.resize(template_image_src, (width, height))
cv2_imshow(template_image)

In [ ]:
target_image_src = cv.imread(target_path)
target_image = cv.resize(target_image_src, (width, height))
cv2_imshow(target_image)

In [ ]:
# add a new dimension to match model's input
template_input = np.expand_dims(template_image.copy(), axis=0)
target_input = np.expand_dims(target_image.copy(), axis=0)

In [ ]:
# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

In [ ]:
if floating_model:
  template_input = (np.float32(template_input) - 127.5) / 127.5
  target_input = (np.float32(target_input) - 127.5) / 127.5

In [ ]:
# Process template image
# Sets the value of the input tensor
interpreter.set_tensor(input_details[0]['index'], template_input)
# Runs the computation
interpreter.invoke()
# Extract output data from the interpreter
template_output_data = interpreter.get_tensor(output_details[0]['index'])
template_offset_data = interpreter.get_tensor(output_details[1]['index'])
# Getting rid of the extra dimension
template_heatmaps = np.squeeze(template_output_data)
template_offsets = np.squeeze(template_offset_data)
print("template_heatmaps' shape:", template_heatmaps.shape)
print("template_offsets' shape:", template_offsets.shape)

In [ ]:
# Process target image
interpreter.set_tensor(input_details[0]['index'], target_input)
interpreter.invoke()
target_output_data = interpreter.get_tensor(output_details[0]['index'])
target_offset_data = interpreter.get_tensor(output_details[1]['index'])
target_heatmaps = np.squeeze(target_output_data)
target_offsets = np.squeeze(target_offset_data)

In [ ]:
def parse_output(heatmap_data,offset_data, threshold):

  joint_num = heatmap_data.shape[-1]
  pose_kps = np.zeros((joint_num,3), np.uint32)

  for i in range(heatmap_data.shape[-1]):

      joint_heatmap = heatmap_data[...,i]
      max_val_pos = np.squeeze(np.argwhere(joint_heatmap==np.max(joint_heatmap)))
      remap_pos = np.array(max_val_pos/8*257,dtype=np.int32)
      pose_kps[i,0] = int(remap_pos[0] + offset_data[max_val_pos[0],max_val_pos[1],i])
      pose_kps[i,1] = int(remap_pos[1] + offset_data[max_val_pos[0],max_val_pos[1],i+joint_num])
      max_prob = np.max(joint_heatmap)

      if max_prob > threshold:
        if pose_kps[i,0] < 257 and pose_kps[i,1] < 257:
          pose_kps[i,2] = 1

  return pose_kps

In [ ]:
def draw_kps(show_img,kps, ratio=None):
    for i in range(5,kps.shape[0]):
      if kps[i,2]:
        if isinstance(ratio, tuple):
          cv.circle(show_img,(int(round(kps[i,1]*ratio[1])),int(round(kps[i,0]*ratio[0]))),2,(0,255,255),round(int(1*ratio[1])))
          continue
        cv.circle(show_img,(kps[i,1],kps[i,0]),2,(0,255,255),-1)
    return show_img

In [ ]:
template_show = np.squeeze((template_input.copy()*127.5+127.5)/255.0)
template_show = np.array(template_show*255,np.uint8)
template_kps = parse_output(template_heatmaps,template_offsets,0.3)
cv2_imshow(draw_kps(template_show.copy(),template_kps))

In [ ]:
target_show = np.squeeze((target_input.copy()*127.5+127.5)/255.0)
target_show = np.array(target_show*255,np.uint8)
target_kps = parse_output(target_heatmaps,target_offsets,0.3)
cv2_imshow(draw_kps(target_show.copy(),target_kps))

In [ ]:
parts_to_compare = [(5,6),(5,7),(6,8),(7,9),(8,10),(11,12),(5,11),(6,12),(11,13),(12,14),(13,15),(14,16)]

In [ ]:
def calc_angle(p1, p2):

  angle = math.atan2(- int(p2[0]) + int(p1[0]), int(p2[1]) - int(p1[1])) * 180.0 / np.pi
  
  return round(angle)

In [ ]:
template_values = []
for part in parts_to_compare:
  template_values.append(calc_angle(template_kps[part[0]][:2], template_kps[part[1]][:2]))
template_values

NameError: ignored

In [ ]:
target_values = []
for part in parts_to_compare:
  target_values.append(calc_angle(target_kps[part[0]][:2], target_kps[part[1]][:2]))
target_values

[(178, 47),
 (-28, 32),
 (-150, 42),
 (-93, 18),
 (-61, 39),
 (-178, 37),
 (-92, 63),
 (-83, 66),
 (-89, 54),
 (-105, 58),
 (-94, 55),
 (-119, 52)]

In [ ]:
def calc_percent_deviation(template_values, target_values):
  frameResults = []

  for i in range(len(template_values)):
    score = abs(abs(target_values[i]) - abs(template_values[i])) / ((abs(template_values[i]) + abs(target_values[i])) / 2)
    frameResults.append(score)

  return frameResults


In [ ]:
def calc_overall_score(devs):
  sum = 0
  for d in devs:
    sum += d

  return (sum/len(devs))

In [ ]:
deviations = calc_percent_deviation(template_values, target_values)
print('Show percent deviations:')
deviations

Show percent deviations:


[2.0,
 0.42105263157894735,
 0.597194388777555,
 1.96661101836394,
 1.5667046750285063,
 0.7924528301886793,
 1.2079207920792079,
 0.9326145552560645,
 1.6837209302325582,
 1.4977578475336322,
 0.46153846153846145,
 1.0140656396517078]

In [ ]:
overall_dev = calc_overall_score(deviations)
print(str(overall_dev))

1.1784694808524385
